# Data Collection: มาเก็บรวบรวมข้อมูลจากแหล่งต่าง ๆ (DB & REST API) กันเถอะ!

# Get data from MySQL database

## Install PyMySQL 
ซึ่งเป็น package สำหรับเชื่อมต่อ MySQL database

In [0]:
! pip install pymysql; print('hi')

## Config DB credential
ಥ_ಥ  คำเตือน: Cell ด้านล่างนี้เป็นความลับสุดยอด เขียนขึ้นมาเพื่อให้เห็นวิธีการในการเชื่อมต่อ database เท่านั้น **ห้ามเอา "plain text"แบบนี้ไปใช้ในชีวิตจริง**

มีข้อปฏิบัติในการเก็บรักษาไฟล์ที่เป็นความลับ (secret) ดังนี้
*   **ห้าม**เขียน credential (ความลับ) ลงมาใน code ตรง ๆ
*   **ห้าม** commit credential ลง Git เด็ดขาด
*   credential ควรเป็น environment variable / ไฟล์ .env / หรือ config file ที่เหมาะสม
*   **ห้าม** commit config file หรือ .env ไฟล์ดังกล่าวขึ้น Git ด้วย
*   ควรใช้ระบบ secret management เพื่อเก็บ credential อย่างปลอดภัย เช่น Vault หรือ Secret Manager
*   การขัดต่อคำแนะนำตามที่กล่าวมาถือว่าบาป



In [0]:
# TODO: ยอมเป็นคนบาปเพื่อการเรียนรู้
class Config:
  MYSQL_HOST = 
  MYSQL_PORT = 3306  # default port ของ MySQL คือ 3306
  MYSQL_USER = 
  MYSQL_PASSWORD = 
  MYSQL_DB = 
  MYSQL_CHARSET = 'utf8mb4'

## Connect to DB
หลังจากที่มี Credential ของ database แล้วก็ connectโลด

In [0]:
import pymysql.cursors

# Connect to the database
connection = pymysql.connect(host=Config.MYSQL_HOST,
                             port=Config.MYSQL_PORT,
                             user=Config.MYSQL_USER,
                             password=Config.MYSQL_PASSWORD,
                             db=Config.MYSQL_DB,
                             charset=Config.MYSQL_CHARSET,
                             cursorclass=pymysql.cursors.DictCursor)

ตัวแปร connection นี้ เราได้ connect ต่อเข้ากับ database เอาไว้แล้ว
## List Tables
เรามาลองดูกันว่ามี table อะไรในนั้นบ้าง

In [0]:
# list all tables
cursor = connection.cursor()
cursor.execute("show tables;")
tables = cursor.fetchall()
cursor.close()
print(tables)

จากโค้ดตัวอย่างด้านบนจะเห็นได้ว่า การคิวรี่ database ทุกครั้ง เราจะต้องสร้าง `cursor` ขึ้นมาเพื่อ query SQL นั้น แล้วก็ปิด cursor ทุกครั้งหลังจบ 

(เหตุผลเป็นไปตามการออกแบบ database สามารถ[อ่านเกี่ยวกับ cursor เพิ่มเติมได้ที่นี่](https://en.wikipedia.org/wiki/Cursor_(databases)))

ดังนั้น จึงนิยมใช้คำสั่ง `with` ในการจัดการสร้าง cursor ขึ้นมา เมื่อจบคำสั่ง cursor จะถูก close ไปเองโดยอัตโนมัติเมื่อออกนอก scope ของ `with`

༼ つ ◕_◕ ༽つ *ขอความกรุณาไม่ drop database หรือลบ data ออก เพราะ database นี้ใช้กันหลายคน :-)*

## Query Table

In [0]:
# ใข้ with statement แทน cursor.close()
# TODO: มาลอง Query ข้อมูลจาก table online_retail ดูกัน

with connection.cursor() as cursor:
  # Read a single record
  sql = "-- ༼ つ ◕_◕ ༽つ remove me. I'm a comment"
  cursor.execute(sql)
  result = cursor.fetchall()

print("number of rows: ", len(result), "⁀⊙﹏☉⁀")

Row เยอะแบบนี้ print ออกมาดูไม่ได้

ใช้งานลำบากอีก ขอแนะนำว่า `Pandas` ช่วยคุณได้ ʕ•́ᴥ•̀ʔ

## Convert to Pandas

In [0]:
import pandas as pd

In [0]:
retail = pd.DataFrame(result_retail)

อ่าว เสร็จแล้วเหรอ ง่ายจัง ( ❛ ᴗ ❛ )

เราก็ได้ data ของเรามาแล้ว



---


# Get data from REST API

หลังจากต่อกับ Database แล้ว ก็มายิ่ง REST API กัน

Package `requests` ใช้สำหรับการยิง REST API

(โดยปกติต้อง install package นี้เพิ่มเติม แต่ colab pre-install ไว้อยู่แล้ว)

วิธีการ install: `pip install requests`

In [0]:
import requests

ลองคลิกดูผลลัพธ์ผ่าน web browser ได้ [Currency conversion API](https://de-training-2020-7au6fmnprq-de.a.run.app/currency_gbp/all) 

ผลลัพธ์ที่ return กลับมาจะเป็นประเภท JSON
จึงต้องใช้ package `json` (built-in) เพื่อโหลดข้อมูลเป็น dictionary

การที่เราสามารถยิง request และ output ออกมาได้เลยโดยที่ไม่ต้องสร้าง payload เพิ่ม ดังตัวอย่างนี้ เรียกว่า HTTP GET (ในกรณีอื่น ๆ สามารถเพิ่ม arguement หรือ query string เข้าไปใน URL ได้)

## Requests library
สามารถศึกษาวิธีการสร้าง request และการใช้งาน package `requests` [ได้ที่นี่](https://requests.readthedocs.io/en/master/)

In [0]:
url = "https://de-training-2020-7au6fmnprq-de.a.run.app/currency_gbp/all"
# TODO: ลองศึกษาวิธีการใช้งาน package requests จากลิ้งค์ด้านบน แล้วลองเขียนโค้ดเพื่อ call URL นี้



มาดูประเภทข้อมูลกัน

In [0]:
type(response.text)

## JSON loading
แต่ JSON ที่เราเห็นมันเป็นเพียงแค่ string

เราจะแปลง string ให้กลายเป็น dictionary!

เราเลยต้อง import `json` (built-in package)

In [0]:
import json

มาดูวิธีการใช้เลยดีกว่า สามารถกดดู help แบบนี้ได้ สำหรับ Jupyter notebook สะดวกสุด ๆ

In [0]:
? json

In [0]:
# TODO: ลองอ่าน string JSON ที่ได้รับ ให้กลายเป็น dictionary
# result =

มาเช็คประเภทข้อมูล

In [0]:
print(type(result_conversion_rate))
assert isinstance(result_conversion_rate, dict)

 ## Convert to Pandas
 แปลงกันอีกครั้งหนึ่ง ʕ•́ᴥ•̀ʔ

In [0]:
conversion_rate = pd.DataFrame.from_dict(result_conversion_rate)

In [0]:
conversion_rate[:3]

In [0]:
conversion_rate = conversion_rate.reset_index().rename(columns={"index":"date"})
conversion_rate[:3]

In [0]:
conversion_rate[:3]

ที่ต้องใช้ `.from_dict()` เพราะว่า ข้อมูลมี key ที่แตกต่างกันออกไป คือ index ที่เป็นเวลา เพราะฉะนั้น `.from_dict()` จึงเข้ามาช่วนการโหลดข้อมูลลักษณะแบบนี้

# Join the data

ในตอนนี้เราจะนำข้อมูลการซื้อขายและข้อมูล Rate การแปลงค่าเงิน เราจะรวมข้อมูลจากทั้งสอง Dataframe มารวมกัน

เราจะนำข้อมูลจากทั้งสองมารวมกันผ่าน column InvoiceDate ใน retail และ date ใน conversion_rate 

แต่ถ้าสังเกตดี ๆ แล้วจะพบว่า InvoiceDate ใน retail จะเก็บข้อมูลในรูปแบบ datetime ส่วน date ใน conversion_rate จะเก็บข้อมูลในรูปแบบ timestamp ที่ส่วนเวลาเป็นเวลา 00:00:00 ทั้งหมด

In [0]:
# ก็อปปี้ column InvoiceDate เก็บเอาไว้ เผื่อได้ใช้ในอนาคต (อันใกล้) --> ไม่งั้นข้อมูล timestamp ของเราจะหายไป ಥ_ಥ
retail['InvoiceTimestamp'] = retail['InvoiceDate']

In [0]:
# แปลงให้ InvoiceDate ใน retail กับ date ใน conversion_rate มีเฉพาะส่วน date ก่อน
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate']).dt.date
conversion_rate['date'] = pd.to_datetime(conversion_rate['date']).dt.date

In [0]:
# รวม 2 dataframe เข้าด้วยกันด้วยคำสั่ง merge
final_df = retail.merge(conversion_rate, how="left", left_on="InvoiceDate", right_on="date")
final_df

พอ join ข้อมูลได้แล้ว เราก็ มา คูณ currency conversion กัน (UnitPrice * Rate)

In [0]:
# TODO: เพิ่ม column 'THBPrice' ที่เกิดจาก column UnitPrice * Rate
# Hint: ลองดู apply function ของ pandas



## Save to CSV

In [0]:
# TODO: save "to csv" file



<== กด ไอคอนรูป ไฟล์ ที่แถบด้านซ้ายเพื่อดูไฟล์ที่เซฟอยู่ใน directory :)

ลองกด Download มาดูได้

In [0]:
print("== End of Workshop 1 ʕ•́ᴥ•̀ʔっ♡ ==")